# HandsOn-11
---
---

### 1. "The basic" Binary Search Tree; this is the one that can be unbalanced



In [19]:
class BSTNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None

class BST:
    def insert(self, root, key):
        if root is None:
            return BSTNode(key)
        if key < root.key:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)
        return root

    def delete(self, root, key):
        if root is None:
            return root
        if key < root.key:
            root.left = self.delete(root.left, key)
        elif key > root.key:
            root.right = self.delete(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left
            temp = self.min_value_node(root.right)
            root.key = temp.key
            root.right = self.delete(root.right, temp.key)
        return root

    def search(self, root, key):
        if root is None or root.key == key:
            return root
        return self.search(root.left, key) if key < root.key else self.search(root.right, key)

    def inorder(self, root):
        return self.inorder(root.left) + [root.key] + self.inorder(root.right) if root else []

    def min_value_node(self, node):
        current = node
        while current.left:
            current = current.left
        return current


In [21]:
# Test BST
bst = BST()
root = None
values = [50, 30, 20, 40, 70, 60, 80]
for v in values:
    root = bst.insert(root, v)

print("BST Inorder:", bst.inorder(root))
root = bst.delete(root, 20)
print("After deleting 20:", bst.inorder(root))
root = bst.delete(root, 50)
print("After deleting root 50:", bst.inorder(root))


BST Inorder: [20, 30, 40, 50, 60, 70, 80]
After deleting 20: [30, 40, 50, 60, 70, 80]
After deleting root 50: [30, 40, 60, 70, 80]


---
---

### 2. Red Black Tree



In [40]:
class RBNode:
    def __init__(self, key):
        self.key = key
        self.parent = None
        self.left = None
        self.right = None
        self.color = 1  # 1=red, 0=black

class RedBlackTree:
    def __init__(self):
        self.NIL = RBNode(0)
        self.NIL.color = 0
        self.NIL.left = None
        self.NIL.right = None
        self.root = self.NIL

    def insert(self, key):
        node = RBNode(key)
        node.key = key
        node.parent = None
        node.left = self.NIL
        node.right = self.NIL
        node.color = 1  # New node is always red

        y = None
        x = self.root

        while x != self.NIL:
            y = x
            if node.key < x.key:
                x = x.left
            else:
                x = x.right

        node.parent = y
        if y is None:
            self.root = node
        elif node.key < y.key:
            y.left = node
        else:
            y.right = node

        if node.parent is None:
            node.color = 0
            return

        if node.parent.parent is None:
            return

        self.fix_insert(node)

    def fix_insert(self, node):
        while node.parent is not None and node.parent.color == 1:
            if node.parent == node.parent.parent.right:
                u = node.parent.parent.left
                if u is not None and u.color == 1:
                    u.color = 0
                    node.parent.color = 0
                    node.parent.parent.color = 1
                    node = node.parent.parent
                else:
                    if node == node.parent.left:
                        node = node.parent
                        self.right_rotate(node)
                    node.parent.color = 0
                    node.parent.parent.color = 1
                    self.left_rotate(node.parent.parent)
            else:
                u = node.parent.parent.right
                if u is not None and u.color == 1:
                    u.color = 0
                    node.parent.color = 0
                    node.parent.parent.color = 1
                    node = node.parent.parent
                else:
                    if node == node.parent.right:
                        node = node.parent
                        self.left_rotate(node)
                    node.parent.color = 0
                    node.parent.parent.color = 1
                    self.right_rotate(node.parent.parent)
            if node == self.root:
                break
        self.root.color = 0

    def left_rotate(self, x):
        y = x.right
        x.right = y.left
        if y.left != self.NIL:
            y.left.parent = x

        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.left:
            x.parent.left = y
        else:
            x.parent.right = y

        y.left = x
        x.parent = y

    def right_rotate(self, x):
        y = x.left
        x.left = y.right
        if y.right != self.NIL:
            y.right.parent = x

        y.parent = x.parent
        if x.parent is None:
            self.root = y
        elif x == x.parent.right:
            x.parent.right = y
        else:
            x.parent.left = y

        y.right = x
        x.parent = y

    def __rb_transplant(self, u, v):
        if u.parent is None:
            self.root = v
        elif u == u.parent.left:
            u.parent.left = v
        else:
            u.parent.right = v
        v.parent = u.parent

    def delete(self, key):
        node = self.root
        while node != self.NIL and node.key != key:
            if key < node.key:
                node = node.left
            else:
                node = node.right

        if node == self.NIL:
            return

        y = node
        y_original_color = y.color
        if node.left == self.NIL:
            x = node.right
            self.__rb_transplant(node, node.right)
        elif node.right == self.NIL:
            x = node.left
            self.__rb_transplant(node, node.left)
        else:
            y = self.minimum(node.right)
            y_original_color = y.color
            x = y.right
            if y.parent == node:
                x.parent = y
            else:
                self.__rb_transplant(y, y.right)
                y.right = node.right
                y.right.parent = y

            self.__rb_transplant(node, y)
            y.left = node.left
            y.left.parent = y
            y.color = node.color

        if y_original_color == 0:
            self.fix_delete(x)

    def fix_delete(self, x):
        while x != self.root and x.color == 0:
            if x == x.parent.left:
                s = x.parent.right
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.left_rotate(x.parent)
                    s = x.parent.right

                if s.left.color == 0 and s.right.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.right.color == 0:
                        s.left.color = 0
                        s.color = 1
                        self.right_rotate(s)
                        s = x.parent.right

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.right.color = 0
                    self.left_rotate(x.parent)
                    x = self.root
            else:
                s = x.parent.left
                if s.color == 1:
                    s.color = 0
                    x.parent.color = 1
                    self.right_rotate(x.parent)
                    s = x.parent.left

                if s.right.color == 0 and s.left.color == 0:
                    s.color = 1
                    x = x.parent
                else:
                    if s.left.color == 0:
                        s.right.color = 0
                        s.color = 1
                        self.left_rotate(s)
                        s = x.parent.left

                    s.color = x.parent.color
                    x.parent.color = 0
                    s.left.color = 0
                    self.right_rotate(x.parent)
                    x = self.root
        x.color = 0

    def minimum(self, node):
        while node.left != self.NIL:
            node = node.left
        return node

    def print_tree(self, node, indent, last):
        if node != self.NIL:
            print(indent, end='')
            if last:
                print("R----", end='')
                indent += "     "
            else:
                print("L----", end='')
                indent += "|    "
            print(f"{node.key}({'RED' if node.color else 'BLACK'})")
            self.print_tree(node.left, indent, False)
            self.print_tree(node.right, indent, True)


In [44]:
# Test RBT
rbt = RedBlackTree()
nums = [7, 3, 18, 10, 22, 8, 11]
for num in nums:
    rbt.insert(num)
print("\nRed-Black Tree Structure:")
rbt.print_tree(rbt.root, "", True)



Red-Black Tree Structure:
R----7(BLACK)
     L----3(BLACK)
     R----18(RED)
          L----10(BLACK)
          |    L----8(RED)
          |    R----11(RED)
          R----22(BLACK)



----
---

### 3. AVL Tree

In [46]:

class AVLNode:
    def __init__(self, key):
        self.key = key
        self.left = None
        self.right = None
        self.height = 1

class AVLTree:
    def insert(self, root, key):
        if not root:
            return AVLNode(key)
        elif key < root.key:
            root.left = self.insert(root.left, key)
        else:
            root.right = self.insert(root.right, key)

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))
        balance = self.get_balance(root)

        if balance > 1 and key < root.left.key:
            return self.right_rotate(root)

        if balance < -1 and key > root.right.key:
            return self.left_rotate(root)

        if balance > 1 and key > root.left.key:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and key < root.right.key:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def delete(self, root, key):
        if not root:
            return root

        elif key < root.key:
            root.left = self.delete(root.left, key)
        elif key > root.key:
            root.right = self.delete(root.right, key)
        else:
            if root.left is None:
                return root.right
            elif root.right is None:
                return root.left

            temp = self.min_value_node(root.right)
            root.key = temp.key
            root.right = self.delete(root.right, temp.key)

        if root is None:
            return root

        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        balance = self.get_balance(root)

        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def left_rotate(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def right_rotate(self, z):
        y = z.left
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))
        return y

    def get_height(self, root):
        return root.height if root else 0

    def get_balance(self, root):
        return (self.get_height(root.left) - self.get_height(root.right)) if root else 0

    def min_value_node(self, root):
        current = root
        while current.left:
            current = current.left
        return current

    def preorder(self, root):
        return [root.key] + self.preorder(root.left) + self.preorder(root.right) if root else []


In [48]:
# Test AVL
avl = AVLTree()
root = None
for v in [10, 20, 30, 40, 50, 25]:
    root = avl.insert(root, v)
print("\nAVL Preorder:", avl.preorder(root))
print("AVL Height:", root.height)
root = avl.delete(root, 40)
print("After deleting 40:", avl.preorder(root))


AVL Preorder: [30, 20, 10, 25, 40, 50]
AVL Height: 3
After deleting 40: [30, 20, 10, 25, 50]


----
---

### Key Features:
**BST:**
- Full insert/delete/search
- Inorder traversal validation

**Red-Black Tree:**
- Color balancing
- Insertion fix
- Visual tree printing

**AVL Tree:**
- Automatic rebalancing
- All rotation cases handled
- Height maintenance

**The test cases demonstrate:**
- BST property maintenance
- RBT color rules
- AVL balance factors
- Proper deletion handling for all cases

---
---